In [356]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go

In [ ]:
def rolling_window(all_overs):
    batsman = []
    window = 0
    start = 0
    for index in range(0, len(all_overs)):
    #     print(index, " : ", type())
        v = set()
        if(window >= 5):
            start = index - 5
            for i in range(start, index+1):
                v.update(all_overs[i])
        else:
            window += 1
            v = np.nan
        batsman.append(v)
    return batsman

In [357]:
def transformFile(filename):
    df_raw = pd.read_csv('dataset/individual_match/'+filename)
    df = df_raw[['match_id', 'venue', 'innings', 'batting_team', 'bowling_team', 'ball', 'striker', 'non_striker', 
         'bowler', 'runs_off_bat', 'extras']].copy()
    
    #Score
    df['score'] = df['runs_off_bat'] + df['extras']
    #Over
    df['over'] = df['ball'].map(np.floor).round(0).astype('int')
    
    feats = ['match_id','venue', 'innings', 'batting_team', 'bowling_team', 'over'] 

    # Innings 1
    innings1 = df[df['innings'] == 1]
    batsmen_per_over = innings1.groupby(feats)['striker', 'non_striker'].apply(lambda x: set(np.unique(x)))
    bowlers_per_over = innings1.groupby(feats)['bowler'].apply(lambda x: set(np.unique(x)))
    runs = innings1.groupby(feats)['score'].sum().cumsum()

    fd_inn1 = pd.DataFrame({
        'batsmen':rolling_window(batsmen_per_over),
        'bowlers':rolling_window(bowlers_per_over),
        'runs': runs}).reset_index()
        
    # Innings 2    
    innings2 = df[df['innings'] == 2]
    batsmen_per_over = innings2.groupby(feats)['striker', 'non_striker'].apply(lambda x: set(np.unique(x)))
    bowlers_per_over = innings2.groupby(feats)['bowler'].apply(lambda x: set(np.unique(x)))
    runs = innings2.groupby(feats)['score'].sum().cumsum()
    
    fd_inn2 = pd.DataFrame({
        'batsmen':rolling_window(batsmen_per_over),
        'bowlers':rolling_window(bowlers_per_over),
        'runs': runs}).reset_index()
    
    fd_inn1.append(fd_inn2, ignore_index = True)
    fd_inn1.to_csv('dataset/individual_match_mod/mod_'+filename)
    
    